---
# Grouping for Aggregation, Filtration, and Transformation
---

In [1]:
import numpy as np
import pandas as pd

## Defining an aggregation
we examine the flights dataset and perform the simplest aggregation involving only a single grouping column, a single aggregating column, and a single aggregating function.
We will find the average arrival delay for each airline.

In [2]:
flights = pd.read_csv('./flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


Define the grouping columns (AIRLINE), aggregating columns (ARR_DELAY), and aggregating functions (mean). Place the grouping column in the .groupby method and then call the .agg method with a dictionary pairing the aggregating column with its aggregating function. If you pass in a dictionary, it returns back a DataFrame instance

In [3]:
(
    flights.groupby(['AIRLINE'])
    .agg({'ARR_DELAY': 'mean'})
)

,ARR_DELAY
AIRLINE,
AA,5.542661
AS,-0.833333
B6,8.692593
DL,0.339691
EV,7.034580
F9,13.630651
HA,4.972973
MQ,6.860591
NK,18.436070


Alternatively, we can place the aggregating column in the index operator and then pass the aggregating function as a string to `.agg`. This will return a Series:

In [8]:
(
    flights.groupby('AIRLINE')
    ['ARR_DELAY'].agg('mean')
 
)

AIRLINE
AA     5.542661
AS    -0.833333
B6     8.692593
DL     0.339691
EV     7.034580
F9    13.630651
HA     4.972973
MQ     6.860591
NK    18.436070
OO     7.593463
UA     7.765755
US     1.681105
VX     5.348884
WN     6.397353
Name: ARR_DELAY, dtype: float64

With numpy mean

In [5]:
(
    flights.groupby('AIRLINE')
    ['ARR_DELAY'].agg(np.mean)
    .head()
)

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

It's possible to skip the agg method altogether in this case and use the code in text
method directly.

In [6]:
(
    flights.groupby('AIRLINE')
    ['ARR_DELAY'].mean()
    .head()
)

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

In [7]:
flights.groupby('AIRLINE')['ARR_DELAY'].agg([np.std]).head()

,std
AIRLINE,
AA,43.323160
AS,31.168354
B6,40.221718
DL,32.299471
EV,36.682336


## Grouping and aggregating with multiple columns and functions

As usual with any kind of grouping operation, it helps to identify the **three** components: 
-  the grouping columns, 
-  aggregating columns, 
-  and aggregating functions.

`.groupby` method by answering the following queries:
-  Finding the number of canceled flights for every airline per weekday
- Finding the number and percentage of canceled and diverted flights for every airline per weekday
-  For each origin and destination, finding the total number of flights, the number and percentage of canceled flights, and the average and variance of the airtime

In [15]:
flights.groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED'].agg('sum')

AIRLINE  WEEKDAY
AA       1          41
         2           9
         3          16
         4          20
         5          18
                    ..
WN       3          18
         4          10
         5           7
         6          10
         7           7
Name: CANCELLED, Length: 98, dtype: int64

In [18]:
flights.groupby(['AIRLINE', 'WEEKDAY'])[['CANCELLED', 'DIVERTED']].agg(['sum', 'mean'])

CANCELLED           DIVERTED          
                      sum      mean      sum      mean
AIRLINE WEEKDAY                                       
AA      1              41  0.032106        6  0.004699
        2               9  0.007341        2  0.001631
        3              16  0.011949        2  0.001494
        4              20  0.015004        5  0.003751
        5              18  0.014151        1  0.000786
...                   ...       ...      ...       ...
WN      3              18  0.014118        2  0.001569
        4              10  0.007911        4  0.003165
        5               7  0.005828        0  0.000000
        6              10  0.010132        3  0.003040
        7               7  0.006066        3  0.002600

[98 rows x 4 columns]

In [19]:
(
    flights.groupby(['ORG_AIR', 'DEST_AIR'])
    .agg(
        {
            'CANCELLED':['sum', 'mean', 'size'],
            'AIR_TIME': ['mean', 'var']
        })
 )

CANCELLED                   AIR_TIME            
                       sum      mean size        mean         var
ORG_AIR DEST_AIR                                                 
ATL     ABE              0  0.000000   31   96.387097   45.778495
        ABQ              0  0.000000   16  170.500000   87.866667
        ABY              0  0.000000   19   28.578947    6.590643
        ACY              0  0.000000    6   91.333333   11.466667
        AEX              0  0.000000   40   78.725000   47.332692
...                    ...       ...  ...         ...         ...
SFO     SNA              4  0.032787  122   64.059322   11.338331
        STL              0  0.000000   20  198.900000  101.042105
        SUN              0  0.000000   10   78.000000   25.777778
        TUS              0  0.000000   20  100.200000   35.221053
        XNA              0  0.000000    2  173.500000    0.500000

[1130 rows x 5 columns]

To flatten the columns, we can use the `.to_flat_index` method

In [21]:
res = (
    flights.groupby(['ORG_AIR', 'DEST_AIR']).agg(
        {
            'CANCELLED':['sum', 'mean', 'size'],
            'AIR_TIME': ['mean', 'var']
        }
    )
)
# res.columns
res.columns = ['_'.join(x) for x in res.columns.to_flat_index()]
res

CANCELLED_sum  CANCELLED_mean  ...  AIR_TIME_mean  AIR_TIME_var
ORG_AIR DEST_AIR                                 ...                             
ATL     ABE                   0        0.000000  ...      96.387097     45.778495
        ABQ                   0        0.000000  ...     170.500000     87.866667
        ABY                   0        0.000000  ...      28.578947      6.590643
        ACY                   0        0.000000  ...      91.333333     11.466667
        AEX                   0        0.000000  ...      78.725000     47.332692
...                         ...             ...  ...            ...           ...
SFO     SNA                   4        0.032787  ...      64.059322     11.338331
        STL                   0        0.000000  ...     198.900000    101.042105
        SUN                   0        0.000000  ...      78.000000     25.777778
        TUS                   0        0.000000  ...     100.200000     35.221053
        XNA                   0        0.000000  ...     173.500000      0.500000

[1130 rows x 5 columns]

An eleguant method to flatten is to chaining with `.pipe` method

In [22]:
def flatten_cols(df):
  df.columns = ['_'.join(x) for x in df.columns.to_flat_index()]
  return df

In [25]:
res = (
    flights.groupby(['ORG_AIR', 'DEST_AIR']).agg(
        {
            'CANCELLED':['sum', 'mean', 'size'],
            'AIR_TIME': ['mean', 'var']
        }
    ).pipe(flatten_cols)
)

res

CANCELLED_sum  CANCELLED_mean  ...  AIR_TIME_mean  AIR_TIME_var
ORG_AIR DEST_AIR                                 ...                             
ATL     ABE                   0        0.000000  ...      96.387097     45.778495
        ABQ                   0        0.000000  ...     170.500000     87.866667
        ABY                   0        0.000000  ...      28.578947      6.590643
        ACY                   0        0.000000  ...      91.333333     11.466667
        AEX                   0        0.000000  ...      78.725000     47.332692
...                         ...             ...  ...            ...           ...
SFO     SNA                   4        0.032787  ...      64.059322     11.338331
        STL                   0        0.000000  ...     198.900000    101.042105
        SUN                   0        0.000000  ...      78.000000     25.777778
        TUS                   0        0.000000  ...     100.200000     35.221053
        XNA                   0        0.000000  ...     173.500000      0.500000

[1130 rows x 5 columns]

Be aware that when grouping with multiple columns, pandas creates a hierarchical index, or multi-index. In the preceding example, it returned 1,130 rows. However, if one of the columns that we group by is categorical (and has a category type, not an object type), then pandas will create a Cartesian product of all combinations for each level. In this case, it returns 2,710 rows. However, if you have categorical columns with higher cardinality, you can get many more values

In [26]:
res = (
    flights.assign(
        ORG_AIR = flights.ORG_AIR.astype('category')
    ).groupby(['ORG_AIR', 'DEST_AIR']).agg(
        {
            'CANCELLED':['sum', 'mean', 'size'],
            'AIR_TIME': ['mean', 'var']
        }
    ).pipe(flatten_cols)
)

In [27]:
res

CANCELLED_sum  CANCELLED_mean  ...  AIR_TIME_mean  AIR_TIME_var
ORG_AIR DEST_AIR                                 ...                             
ATL     ABE                   0             0.0  ...      96.387097     45.778495
        ABI                   0             NaN  ...            NaN           NaN
        ABQ                   0             0.0  ...     170.500000     87.866667
        ABR                   0             NaN  ...            NaN           NaN
        ABY                   0             0.0  ...      28.578947      6.590643
...                         ...             ...  ...            ...           ...
SFO     TYS                   0             NaN  ...            NaN           NaN
        VLD                   0             NaN  ...            NaN           NaN
        VPS                   0             NaN  ...            NaN           NaN
        XNA                   0             0.0  ...     173.500000      0.500000
        YUM                   0             NaN  ...            NaN           NaN

[2710 rows x 5 columns]

To remedy the combinatoric explosion, use the `observed=True` parameter. This makes the categorical group bys work like grouping with string types, and only shows the observed values and not the Cartesian product

In [28]:
res = (
    flights.assign(
        ORG_AIR = flights.ORG_AIR.astype('category')
    ).groupby(['ORG_AIR', 'DEST_AIR'], observed=True).agg(
        {
            'CANCELLED':['sum', 'mean', 'size'],
            'AIR_TIME': ['mean', 'var']
        }
    ).pipe(flatten_cols)
)

res

CANCELLED_sum  CANCELLED_mean  ...  AIR_TIME_mean  AIR_TIME_var
ORG_AIR DEST_AIR                                 ...                             
LAX     ABQ                   1        0.018182  ...      89.259259     29.403215
        ANC                   0        0.000000  ...     307.428571     78.952381
        ASE                   1        0.038462  ...     102.920000    102.243333
        ATL                   0        0.000000  ...     224.201149    127.155837
        AUS                   0        0.000000  ...     150.537500     57.897310
...                         ...             ...  ...            ...           ...
MSP     TTN                   1        0.125000  ...     124.428571     57.952381
        TUL                   0        0.000000  ...      91.611111     63.075163
        TUS                   0        0.000000  ...     176.000000     32.000000
        TVC                   0        0.000000  ...      56.600000     10.300000
        XNA                   0        0.000000  ...      90.642857    115.939560

[1130 rows x 5 columns]